# 데이터 분석을 위한 추가 전처리
1. 'stos', 'dtos' 제거
2. 'label' 에서 'target 생성 및 label 제거
3. 숫자형 데이터 MinMaxScaling

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset

from scipy.optimize import linear_sum_assignment as linear_assignment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


In [ ]:
dataset=pd.read_csv('/content/drive/MyDrive/캡스톤/dataset/CTU-13(10)_preprocessed.csv')

In [ ]:
# column 명들 소문자로 변환
dataset.columns = dataset.columns.str.lower()

In [ ]:
# 'stos0' 'dtos0' 모두 1.0 의 값을 가지므로 분류에 필요하지 않은 것 같아 삭제 (10번 시나리오에서는)
dataset.drop(['stos0','dtos0'],axis=1,inplace=True)
dataset

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport2,dport3,dport4,state0,state1,state2,totpkts,totbytes,srcbytes,label
0,3587.569824,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,3049,978731,245317,flow=From-Normal-V51-Grill
1,198.072739,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,14,924,462,flow=From-Normal-V51-Grill
2,197.928329,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,14,924,462,flow=From-Normal-V51-Grill
3,0.000399,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,2,400,74,flow=From-Normal-V51-Stribrek
4,0.000400,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,2,400,74,flow=From-Normal-V51-Stribrek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122194,0.000743,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2,196,98,flow=From-Normal-V51-Grill
122195,0.000913,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2,196,98,flow=From-Normal-V51-Grill
122196,0.000414,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,2,244,81,flow=From-Normal-V51-Stribrek
122197,0.000322,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,2,280,81,flow=From-Normal-V51-Stribrek


In [ ]:
# Botnet 과 Normal 구분 -> 정상 0, 비정상 1
def convertLabel(sample):
  if 'Botnet' in sample: return 1
  else: return 0

dataset['target'] = dataset['label'].apply(convertLabel)

In [ ]:
# label feature 제거
dataset.drop(['label'],axis=1,inplace=True)

In [ ]:
# 숫자형 feature -> MinMaxScaling
columns=['dur','totpkts','totbytes','srcbytes']
scaler=MinMaxScaler()
dataset[columns]=scaler.fit_transform(dataset[columns])

## Train data, Test data 나누기
**논문에 나와있는 대로**
1. Train data -> Normal data(40%)
2. Test data -> Normal data(60%) + Abnormal data(60%)

In [ ]:
# 정상 dataset / 비정상 dataset
normal_dataset=dataset[dataset['target']==0]
abnormal_dataset=dataset[dataset['target']==1]

# 정상 dataset -> Train 정상 / Test 정상 나누기 , 40%:60%
normal_dataset = normal_dataset.drop(['target'],axis=1) # 'target' feature drop 
train_normal,test_normal=train_test_split(normal_dataset, test_size=0.6, random_state=42)

In [ ]:
train_normal

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport1,dport2,dport3,dport4,state0,state1,state2,totpkts,totbytes,srcbytes
2157,6.555634e-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.000006,1.326236e-06,5.838319e-07
4221,1.516685e-07,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.000006,2.450653e-06,5.333773e-07
82290,6.102351e-05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.000051,8.353845e-06,3.957083e-06
83840,5.121450e-06,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.000028,2.537147e-06,1.960522e-06
80723,0.000000e+00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,1.203703e-06,1.636171e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88335,4.416719e-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.000006,1.059547e-06,5.405851e-07
5544,6.265741e-02,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.000096,1.303171e-05,8.173646e-06
865,6.344492e-05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.000051,8.353845e-06,3.957083e-06
122145,1.908356e-07,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000006,9.802614e-07,7.063645e-07


In [ ]:
# 비정상 dataset에서 random 으로 60% 추출
test_abnormal = abnormal_dataset.sample(frac=0.6)
test_abnormal = test_abnormal.drop(['target'],axis=1) # 'target' feature drop 

In [ ]:
test_abnormal

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport1,dport2,dport3,dport4,state0,state1,state2,totpkts,totbytes,srcbytes
96068,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000007,0.000008
59206,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000007,0.000008
36381,0.275124,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000142,0.000199,0.000200
47117,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000007,0.000008
25392,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000007,0.000008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67910,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000007,0.000008
10884,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000007,0.000008
70613,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000007,0.000008
46401,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000007,0.000008


# Dataframe 을 pytorch tensor로 변환 

In [ ]:
# pytorch tensor 로 변환 

# determine the supported device
def get_device():
  if torch.cuda.is_available():
    device = torch.device('cuda:0')
  else :
    device = torch.device('cpu') # don 't have GPU 
  return device

# convert a df to tensor to be used in pytorch
def dataframe_to_tensor(df):
   device = get_device()
   return torch.from_numpy(df.values).float().to(device)

In [ ]:
train_normal = dataframe_to_tensor(train_normal)

# Deep K-means
1. layer 수는 5
2. mini-batch size 는 100 
3. 각 layer에서 얼만큼 줄일지 + 활성화 함수는 어떤 것을 사용할지 ?
4. learning rate=0.001 인데 어느 정도로 해야할지?

--> 3차원으로 latent vector 크기 잡아도 될듯

In [ ]:
# train_loader 생성
train_loader = DataLoader(train_normal,batch_size=100,shuffle=True)
train_batch = iter(train_loader).next()
print(train_batch.shape)


torch.Size([100, 27])


In [ ]:
class TestDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len


In [ ]:
# test_loader 생성
test_data = pd.concat([test_normal, test_abnormal]).values
test_label = np.concatenate([np.zeros(9509),np.ones(63811)])

test_data = TestDataset(test_data,test_label)

test_loader = DataLoader(test_data,batch_size=100,shuffle=True)

In [ ]:
class Encoder(nn.Module):
  def __init__(self,latent_size):
    super(Encoder,self).__init__()

    self.encoder = nn.Sequential(
                      nn.Linear(27,16),
                      nn.ReLU(),
                      nn.Linear(16,latent_size),
                      nn.ReLU())
          
    
  def forward(self, x):
    return self.encoder(x)

class Decoder(nn.Module):
  def __init__(self, latent_size):
    super(Decoder,self).__init__()

    self.decoder = nn.Sequential(
                      nn.Linear(latent_size,16),
                      nn.ReLU(),
                      nn.Linear(16,27),
                      nn.Sigmoid())
    
  def forward(self,x):
    return self.decoder(x)

In [ ]:
class Kmeans(nn.Module):
  def __init__(self,num_clusters, latent_size):
    super(Kmeans,self).__init__()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.num_clusters = num_clusters
    self.centroids = nn.Parameter(torch.rand((self.num_clusters,latent_size)).to(device))

  def argminl2distance(self,a,b):
    return torch.argmin(torch.sum((a-b)**2,dim=1),dim=0)

  def forward(self,x):
    y_assign = []
    for m in range(x.size(0)):
      h = x[m].expand(self.num_clusters,-1)
      assign = self.argminl2distance(h, self.centroids)
      y_assign.append(assign.item())
    return y_assign, self.centroids[y_assign]

In [ ]:
def cluster_acc(y_true,y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  D = max(y_pred.max(), y_true.max()) + 1
  w = np.zeros((D,D), dtype = np.int64)
  for i in range(y_pred.size):
    w[y_pred[i],y_true[i]] +=1
  ind = linear_assignment(w.max() - w)
  return sum([w[i,j] for i,j in zip(ind[0],ind[1])]) * 1.0 / y_pred.size

In [ ]:
def evaluation(testloader, encoder, kmeans, device):
  predictions = []
  actual = []

  with torch.no_grad():
    for images, labels in testloader:
      inputs = images.to(device)
      labels = labels.to(device)
      latent_var = encoder(inputs)
      y_pred,_ = kmeans(latent_var)

      predictions+=y_pred
      actual+=labels.cpu().tolist()

  return cluster_acc(actual,predictions)

#모델 학습


In [ ]:
latent_size=4
num_clusters=2

In [ ]:
encoder = Encoder(latent_size).to(device)
decoder = Decoder(latent_size).to(device)
kmeans = Kmeans(num_clusters, latent_size).to(device)
criterion1 = torch.nn.MSELoss()
criterion2 = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters())+list(kmeans.parameters()), lr=0.001)

In [ ]:
T1=100
T2=200
lam = 1e-3
ls = 0.05

In [ ]:
for ep in range(300):
  if ep <= T1:
    alpha = lam/(T2-T1)
  else:
    alpha = lam
  
  # 정상 data(label=0)에 대해서만 학습
  running_loss = 0.0
  for batch in train_loader:
    inputs = batch.to(device)
    optimizer.zero_grad()
    latent_var = encoder(inputs)
    _, centroids = kmeans(latent_var.detach())
    outputs = decoder(latent_var)

    l_rec = criterion1(inputs,outputs)
    l_clt = criterion2(latent_var,centroids)
    loss = l_rec + alpha*l_clt

    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  
  ###
  avg_loss = running_loss/len(train_loader)

  if(ep%10==0):
    print('[%d] Train loss:%.4f' %(ep,avg_loss) )

[0] Train loss:0.2354
[10] Train loss:0.0267


KeyboardInterrupt: ignored

# 학습 후 One class svm ensemble

In [ ]:
encoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(CTU-13(10))/encoder.pth'))
decoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(CTU-13(10))/decoder.pth'))
kmeans.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(CTU-13(10))/kmeans.pth'))

<All keys matched successfully>

In [ ]:
# svm 에 활용할 latent_vectors 생성
# svm 에 활용할 latent_vectors_labels 생성

train_latent_vectors = [] # latent vector 가 list로 계속해서 들어간 형태
train_predict_label= []

for data in train_loader:
  inputs = data.to(device)
  latent_vector= encoder(inputs)
  predict_label, _ = kmeans(latent_vector)

  train_latent_vectors+=latent_vector.cpu().tolist()
  train_predict_label+=predict_label


In [ ]:
# svm 에 활용할 test_latent_vectors 생성
# svm 에 활용할 test_label (실제 label 값) 생성

test_latent_vectors = [] # latent vector 가 list로 계속해서 들어간 형태
test_label= []

for data,labels in test_loader:
  inputs = data.to(device)
  latent_vector= encoder(inputs)

  test_latent_vectors+=latent_vector.cpu().tolist()
  test_label+=labels.cpu().tolist()

# OneSvm Ensemble

In [ ]:
#OneclassSVM Ensemble
from sklearn.svm import OneClassSVM
import numpy as np
import pandas as pd

class OCSVMEnsemble():

  def __init__(self,nu=0.5):
    self.nu = nu
    #self.gamma = gamma

  def fit(self, latent, pred):
    # oneclasssvm 인스턴스가 들어갈 리스트
    self.instance = []
    self.num_cluster = len(np.unique(pred))

    # 경계가 9개 생긴거
    for clu in range(self.num_cluster):
      idx = np.where(pred == clu) # 0-9 까지의 각 그룹에 해당하는 index 도출 
      #해당 군집에 속한 data
      clu_data = latent[idx]
      ocsvm = OneClassSVM(kernel = 'rbf',gamma = 'scale', nu = self.nu).fit(clu_data) # OneClassSVM 을 통한 객체 ocsvm 생성
      self.instance.append(ocsvm)

  # 모든 OneClassSVM 인스턴스를 돌면서
  # inlier -> 0 , outlier -> 1 return 하는 함수
  def predictLabel(self,x):
    for model in self.instance:
      model_predict = model.predict(x) # inliers 1, outliers -1
      if model_predict == 1:
        return 0
    return 1

In [ ]:
# train_latent_vectors, latent_vectors_labels numpy 로 변환
train_latent_vectors=np.array(train_latent_vectors)
train_predict_label=np.array(train_predict_label)

In [ ]:
test_latent_vectors = np.array(test_latent_vectors)
test_label = np.array(test_label)

In [ ]:
# 학습 과정
ocsvm = OCSVMEnsemble(nu=0.3) 
ocsvm.fit(train_latent_vectors, train_predict_label) # train data 에서 latent vector 들 + 가장 가까운 중심의 index

In [ ]:
kmeans.centroids

Parameter containing:
tensor([[ 2.7903e+00,  5.6486e+00,  1.5954e+00,  2.2993e-41],
        [ 4.1532e-01,  2.5053e+00,  2.1851e+00, -2.8137e-40]], device='cuda:0',
       requires_grad=True)

In [ ]:
train_latent_vectors[0:20]

array([[2.9539547 , 5.46568012, 1.47538233, 0.        ],
       [0.68853122, 2.55816174, 1.47827458, 0.        ],
       [2.56933022, 5.76385164, 1.53415704, 0.        ],
       [0.23496504, 2.64593267, 2.18573499, 0.        ],
       [0.23496367, 2.64593148, 2.18573332, 0.        ],
       [0.23496367, 2.64593148, 2.18573332, 0.        ],
       [0.59083688, 2.37246847, 2.28852868, 0.        ],
       [2.66663218, 5.87593985, 1.5162859 , 0.        ],
       [0.5906117 , 2.37210011, 2.28851104, 0.        ],
       [0.59061944, 2.37211227, 2.28851104, 0.        ],
       [0.23496383, 2.64593053, 2.18573475, 0.        ],
       [2.95928216, 5.4718318 , 1.47439528, 0.        ],
       [0.59068316, 2.37221622, 2.28851724, 0.        ],
       [0.23496401, 2.64593101, 2.18573451, 0.        ],
       [0.59061158, 2.37209988, 2.28851151, 0.        ],
       [2.95416045, 5.46588564, 1.47528315, 0.        ],
       [0.23496377, 2.64593077, 2.18573475, 0.        ],
       [2.95394969, 5.46567917,

In [ ]:
train_predict_label[0:10]

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 1])

In [ ]:
test_latent_vectors[10:20]

array([[1.53545737, 3.18143559, 1.60404134, 0.        ],
       [1.53545737, 3.18143559, 1.60404134, 0.        ],
       [2.56360626, 5.75725079, 1.53521156, 0.        ],
       [3.24982238, 4.80489683, 1.92127466, 0.        ],
       [1.53545737, 3.18143559, 1.60404134, 0.        ],
       [1.53545737, 3.18143559, 1.60404134, 0.        ],
       [1.53545737, 3.18143559, 1.60404134, 0.        ],
       [1.53545737, 3.18143559, 1.60404134, 0.        ],
       [1.53545737, 3.18143559, 1.60404134, 0.        ],
       [1.53545737, 3.18143559, 1.60404134, 0.        ]])

In [ ]:
test_label[10:20]

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# test과정에서의 latent vector 들을 넣어가면서 예측하는 과정
y_pred = []
for i in test_latent_vectors:
  i=i.reshape(1,-1)
  pred=ocsvm.predictLabel(i)
  y_pred.append(pred)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(test_label,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.76      0.86      9509
           1       0.97      1.00      0.98     63811

    accuracy                           0.97     73320
   macro avg       0.98      0.88      0.92     73320
weighted avg       0.97      0.97      0.97     73320



In [ ]:
y_pred[10:20]

[1, 1, 0, 1, 1, 1, 1, 1, 1, 1]